### **PyTorchの使い方のメモ**
触りながら学ぶ<br>
[参考] https://qiita.com/north_redwing/items/30f9619f0ee727875250

### > tensorの定義方法

In [3]:
import torch

# 様々なmatrix
x = torch.empty((50000,3)) # 5x3のmatrix < 初期化されてないので注意!
#x = torch.rand((5,3)) # 5x3のrandom matrix
#x = torch.zeros((5,3), dtype=torch.long) # 0(long type)で初期化したmatrix

### > numpyとtensorのリンク

In [2]:
import numpy as np

a = torch.ones(5)
print(a)

# torch > numpy
b = a.numpy()
print(b)

# numpy > torch
c = torch.from_numpy(b)
print(c)

tensor([1., 1., 1., 1., 1.])
[1. 1. 1. 1. 1.]
tensor([1., 1., 1., 1., 1.])


### > GPU(CUDA)を使うには

In [4]:
device = torch.device("cuda")

# tensorを作成するときに device=で指定する
x_on_GPU = torch.rand((50000,3),device=device)

# toを使ってGPUに乗せる
x = torch.rand((50000,3))
x_on_GPU = x.to(device)

### > Tensorが持つ要素
- torch.Tensorには, .require_grad, .gradというattributeが, .backward()というメソッドを持っています.
- この.require_gradをTrueに設定すれば, このtensorに与えられる演算を全て記憶し, .backward()により, 勾配を計算してくれます. その勾配の計算結果は, .gradが保持しています.
- .grad_fnは, 勾配計算のために, 演算に用いた関数を示してくれます.

In [4]:
import torch
x = torch.ones(2, 2, requires_grad=True)
print(x)

y = x + 2
print(y)

# 入力xのrequires_gradをTrueにしたので, yもTrueになっている
print(y.requires_grad)
print(y.grad_fn)


z = y * y * 3
print(z)

out = z.mean()
print(out)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
True
tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)


In [5]:
out.backward()
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


### > NNを実装する
[CNN参考] https://ml4a.github.io/ml4a/jp/convnets/

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

    

# NNを定義して一つだけサンプルを入力してみる
net = Net() #モデル定義 
x = torch.randn(1,1,32,32) # サブサンプル1個の32X32の画像1個だけ生成
pred = net(x) #入力して10次元の出力を得る

print(pred)

tensor([[-0.0217, -0.0225,  0.0179,  0.1118, -0.0884,  0.1217, -0.0210,  0.0145,
          0.0017, -0.0876]], grad_fn=<AddmmBackward>)


### LossとGradientを計算する

In [36]:
torch.manual_seed(0)
pred = net(x)
y = torch.randn(10)  # a dummy target, for example
#print(y)
y = y.view(1, -1)  # make it the same shape as output
#print(y)
criterion = nn.MSELoss()

loss = criterion(pred, y) #ロスを計算する
net.zero_grad() #Gradientの値を初期化する.
loss.backward() #ロスのGradientを計算する

え, lossの購買計算するだけなら, zero_gradなんていらないのでは? と思ったが, どうやらbackward()では勾配が累積するそう.<br>
[参考]https://teratail.com/questions/235205<br>
RNNのように時間方向にロスを累積させたりするネットワークの場合は, この仕様の方が便利なのかも.